# Preliminaries

In [1]:
import torch
if torch.cuda.is_available():
    torch.cuda.set_device(0)
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

print(f"Using device: {device}")

Using device: cpu


# Generating embeddings for graphs 

In [3]:
from embedders import dataloaders
#Load Data
polblogs_dists, polblogs_labels,sub = dataloaders.load("polblogs")

Top CC has 1222 nodes; original graph has 1490 nodes.


In [5]:
# Specify signature - useful to re-initialize the manifold here
from embedders import manifolds
from embedders import coordinate_learning
torch.manual_seed(0)  
signature = [(1, 4)]
pm = manifolds.ProductManifold(signature=signature)
print(pm.name)

# Rescale distances
dists_rescaled = polblogs_dists / polblogs_dists.max()

# Get embedding
coordinate_learning.train_coords(
    pm,
    dists_rescaled,
    device=device,
    burn_in_iterations=100,
    training_iterations=100 * 9,
    learning_rate=1e-1,
    burn_in_learning_rate=1e-2,
    scale_factor_learning_rate=1e-1,
)

h6_polblogs = pm.x_embed.detach().cpu()

S_1.0^4


  0%|          | 0/1000 [00:00<?, ?it/s]

In [7]:
#Evaluation
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from embedders.predictors import tree_new
X_train, X_test, y_train, y_test = train_test_split(h6_polblogs, polblogs_labels.detach().cpu(), test_size=0.2, random_state=0)
pdt = tree_new.ProductSpaceDT(pm=pm, max_depth=3, use_special_dims=True)
pdt.fit(X_train, y_train)
pdt_f1 = f1_score(y_test, pdt.predict(X_test),average="weighted")
print(f"ProductDT\t{pdt_f1*100:.2f}")

ProductDT	80.86


# Generating embeddings for Mixed Curvature VAE 

In [9]:
#Load data
X, y, _ = dataloaders.load("lymphoma")
print(X.shape, y.shape)

#Define Encoder and Decoder
class Encoder(torch.nn.Module):
    def __init__(self, pm):
        super().__init__()
        self.pm = pm
        self.fc1 = torch.nn.Linear(1056, 200)
        self.fc2 = torch.nn.Linear(200, 200)
        self.fc3_z_mean = torch.nn.Linear(200, pm.dim)
        self.fc3_z_logvar = torch.nn.Linear(200, pm.dim)

    def forward(self, x):
        # Hidden layers
        h1 = torch.relu(self.fc1(x))
        h2 = torch.relu(self.fc2(h1))

        # Reparameterization
        z_mean_tangent = self.fc3_z_mean(h2)
        z_logvar = self.fc3_z_logvar(h2)
        z_mean = pm.manifold.expmap(x=pm.mu0, u=z_mean_tangent @ pm.projection_matrix)

        return z_mean, z_logvar


class Decoder(torch.nn.Module):
    def __init__(self, pm):
        super().__init__()
        self.pm = pm
        self.fc1 = torch.nn.Linear(pm.ambient_dim, 200)
        self.fc2 = torch.nn.Linear(200, 200)
        self.fc3 = torch.nn.Linear(200, 1056)

    def forward(self, z):
        # Hidden layers
        h1 = torch.relu(self.fc1(z))
        h2 = torch.relu(self.fc2(h1))

        # Output layer
        x = torch.sigmoid(self.fc3(h2))

        return x

torch.Size([13410, 1056]) torch.Size([13410])


In [11]:
from tqdm.notebook import tqdm
import numpy as np
from embedders import vae
#Specify the hyperparameter
SIGNATURE = [(1, 2), (1, 2)]
N_EPOCHS = 100
BATCH_SIZE = 4_096
BETA = 1.0
N_SAMPLES = 32
TRIALS = 3

# Skopek hyperparamters (aka standard Adam hyperparameters)
LR = 1e-4 
BETA1 = 0.9
BETA2 = 0.999
EPS = 1e-8
CURVATURE_LR = 1e-4


for trial in range(TRIALS):
    # Set seeds
    torch.manual_seed(trial)
    np.random.seed(trial)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=trial)
    pm = manifolds.ProductManifold(SIGNATURE)
    myvae = vae.ProductSpaceVAE(
        pm=pm, encoder=Encoder(pm), decoder=Decoder(pm), beta=BETA, n_samples=N_SAMPLES, device=device
    )
    opt = torch.optim.Adam(
        [
            {"params": myvae.parameters(), "lr": LR * 0.1, "betas": (BETA1, BETA2), "eps": EPS},
            {"params": pm.params(), "lr": 0, "betas": (BETA1, BETA2), "eps":  EPS}
        ]
    )

    # Visualization stuff
    my_tqdm = tqdm(total=N_EPOCHS * len(X_train))

    # Device stuff
    myvae = myvae.to(device)
    
    X_train = X_train.to(device)
    X_test = X_test.to(device)
    pm = pm.to(device)


    # Gradient checking stuff
    def grads_ok(myvae):
        out = True
        for name, param in myvae.named_parameters():
            if param.grad is not None:
                if torch.isnan(param.grad).any():
                    print(f"NaN gradient in {name}")
                    out = False
                if torch.isinf(param.grad).any():
                    print(f"Inf gradient in {name}")
                    out = False
        return out


    losses = []
    for epoch in range(N_EPOCHS):
        # Stop the burn-in
        if epoch == 20:
            opt.param_groups[1]["lr"] = CURVATURE_LR
            opt.param_groups[0]["lr"] = LR
        
        for i in range(0, len(X_train), BATCH_SIZE):
            x_batch = X_train[i : i + BATCH_SIZE]

            elbo, ll, kl = myvae.elbo(x_batch)
            loss = -elbo
            losses.append(loss.item())

            opt.zero_grad()
            loss.backward()


            if torch.isnan(loss):
                print(f"Loss is NaN at iteration {i}")
            elif torch.isinf(loss):
                print(f"Loss is inf at iteration {i}")
            elif grads_ok(myvae):
                opt.step()

            my_tqdm.update(BATCH_SIZE)
            my_tqdm.set_description(f"Epoch {epoch}, loss: {loss.item():.1f}, ll: {ll.item():.1f}, kl: {kl.item():.1f}")
            my_tqdm.set_postfix(
                {f"r{i}": f"{x._log_scale.item():.3f}" for i, x in enumerate(pm.manifold.manifolds)}
            )
    
    # Save the embeddings
    embeddings = []
    for i in range(0, len(X_train), BATCH_SIZE):
        x_batch = X_train[i : i + BATCH_SIZE]
        z_mean, _ = myvae.encoder(x_batch)
        embeddings.append(z_mean.detach().cpu().numpy())

    embeddings = np.concatenate(embeddings)

    # Save the test embeddings
    test_embeddings = []
    for i in range(0, len(X_test), BATCH_SIZE):
        x_batch = X_test[i : i + BATCH_SIZE]
        z_mean, _ = myvae.encoder(x_batch)
        test_embeddings.append(z_mean.detach().cpu().numpy())
    
    test_embeddings = np.concatenate(test_embeddings)
    

  0%|          | 0/1072800 [00:00<?, ?it/s]

  0%|          | 0/1072800 [00:00<?, ?it/s]

  0%|          | 0/1072800 [00:00<?, ?it/s]

# Gaussian Mixture Generator 

In [10]:
from embedders import gaussian_mixture
# Filter out warnings raised when sampling Wishart distribution in Gaussian mixtures
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

#Gaussian mixture generator
pm = manifolds.ProductManifold(signature=[(-1, 2),(2, 3),(0, 5)])
X, y = gaussian_mixture.gaussian_mixture(pm=pm, seed=42, cov_scale_means=1.0, cov_scale_points=1.0)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
#Example of apply gaussian mixture to product space decision tree
pdt = tree_new.ProductSpaceDT(pm=pm, max_depth=3, use_special_dims=True)
pdt.fit(X_train, y_train)
pdt_f1 = f1_score(y_test, pdt.predict(X_test),average="micro")
print(f"ProductDT\t{pdt_f1*100:.2f}")

ProductDT	83.00
